# 1)A) k-nearest  neighbors  regression  model,  and  report  its  validation  set  performance  using  
root mean squared error.

In [213]:
import pandas as pd
import numpy as np

cols = ['symboling', 'normalized-losses', 'make', 'fuel-type',
        'aspiration', 'num-of-doors', 'body-style', 'drive-wheels',
        'engine-location', 'wheel-base', 'length', 'width', 'height',
        'curb-weight', 'engine-type', 'num-of-cylinders', 'engine-size', 'fuel-system', 'bore', 'stroke',
        'compression-ratio', 'horsepower', 'peak-rpm', 'city-mpg', 'highway-mpg', 'price']

missing_val = ['?']
cars = pd.read_csv(r'C:\Users\Amar Raja\Desktop\iitkML\assignment1\KNN\imports-85\imports-85.data',names=cols,sep = ',',na_values=missing_val)


pd.set_option('display.max_columns',30)
pd.set_option('display.max_rows',200)
pd.set_option('display.width',320)



DATASET

In [214]:
cars.head()

,symboling,normalized-losses,make,fuel-type,aspiration,num-of-doors,body-style,drive-wheels,engine-location,wheel-base,length,width,height,curb-weight,engine-type,num-of-cylinders,engine-size,fuel-system,bore,stroke,compression-ratio,horsepower,peak-rpm,city-mpg,highway-mpg,price
0,3,NaN,alfa-romero,gas,std,two,convertible,rwd,front,88.6,168.8,64.1,48.8,2548,dohc,four,130,mpfi,3.47,2.68,9.0,111.0,5000.0,21,27,13495.0
1,3,NaN,alfa-romero,gas,std,two,convertible,rwd,front,88.6,168.8,64.1,48.8,2548,dohc,four,130,mpfi,3.47,2.68,9.0,111.0,5000.0,21,27,16500.0
2,1,NaN,alfa-romero,gas,std,two,hatchback,rwd,front,94.5,171.2,65.5,52.4,2823,ohcv,six,152,mpfi,2.68,3.47,9.0,154.0,5000.0,19,26,16500.0
3,2,164.0,audi,gas,std,four,sedan,fwd,front,99.8,176.6,66.2,54.3,2337,ohc,four,109,mpfi,3.19,3.40,10.0,102.0,5500.0,24,30,13950.0
4,2,164.0,audi,gas,std,four,sedan,4wd,front,99.4,176.6,66.4,54.3,2824,ohc,five,136,mpfi,3.19,3.40,8.0,115.0,5500.0,18,22,17450.0


CHECKING DTYPES OF ALL ATTRIBUTES

In [215]:
cars.dtypes

symboling              int64
normalized-losses    float64
make                  object
fuel-type             object
aspiration            object
num-of-doors          object
body-style            object
drive-wheels          object
engine-location       object
wheel-base           float64
length               float64
width                float64
height               float64
curb-weight            int64
engine-type           object
num-of-cylinders      object
engine-size            int64
fuel-system           object
bore                 float64
stroke               float64
compression-ratio    float64
horsepower           float64
peak-rpm             float64
city-mpg               int64
highway-mpg            int64
price                float64
dtype: object

CHECKING NAN's

In [216]:
cars.isna().sum()

symboling             0
normalized-losses    41
make                  0
fuel-type             0
aspiration            0
num-of-doors          2
body-style            0
drive-wheels          0
engine-location       0
wheel-base            0
length                0
width                 0
height                0
curb-weight           0
engine-type           0
num-of-cylinders      0
engine-size           0
fuel-system           0
bore                  4
stroke                4
compression-ratio     0
horsepower            2
peak-rpm              2
city-mpg              0
highway-mpg           0
price                 4
dtype: int64

In [217]:
#fillna with mean
mean = cars['normalized-losses'].mean()
cars['normalized-losses'].fillna(value=mean,inplace = True)

# interpolate bore stroke
cars[['bore','stroke']] = cars[['bore','stroke']].interpolate()
# cars.loc[50:66,:].head(200)

#fillna with bfill after finding pattern in data
cars[['horsepower','peak-rpm']]=cars[['horsepower','peak-rpm']].fillna(method='bfill')
# cars.iloc[125:135,[0,2,20,21,22,23,24,25]]

# fillna with ffill after finding pattern in data
cars['num-of-doors']=cars['num-of-doors'].fillna(method='ffill')

#replacing string with respective numeric values
# for columns 'num-of-doors' and 'num-of-cylinders'
cars['num-of-doors'] = cars['num-of-doors'].replace(to_replace=['two','four'],value=[2,4])
cars['num-of-doors'] = cars['num-of-doors'].astype('int')

original = ['two','three','four','five','six','eight','twelve']
modf = [2,3,4,5,6,8,12]

cars['num-of-cylinders'] = cars['num-of-cylinders'].replace(to_replace = original,value = modf)
cars['num-of-cylinders'] = cars['num-of-cylinders'].astype('int')

In [218]:
#droping rows with price NAN
cars.dropna(subset=['price'],inplace=True)

Checking if there are no NAN's

In [219]:
cars.isna().sum().sum()

0

changing Dtypes object to Category

In [220]:
cars['make'] = cars['make'].astype('category')
cars['fuel-type'] = cars['make'].astype('category')
cars['body-style']=cars['body-style'].astype('category')
cars['drive-wheels']=cars['drive-wheels'].astype('category')
cars['engine-location']=cars['engine-location'].astype('category')
cars['fuel-system']=cars['fuel-system'].astype('category')
cars['engine-type'] = cars['engine-type'].astype('category')

In [221]:
#finding correalation
correlation = cars.corr()
correlation=correlation['price'].sort_values(ascending=False).to_frame()
correlation

,price
price,1.000000
engine-size,0.872335
curb-weight,0.834415
horsepower,0.808753
width,0.751265
num-of-cylinders,0.708645
length,0.690628
wheel-base,0.584642
bore,0.543437
height,0.135486


#dropping cols with low correlation

In [222]:
cars=cars.drop(['stroke','compression-ratio','num-of-doors','symboling'],axis=1)

In [223]:
cars.columns

Index(['normalized-losses', 'make', 'fuel-type', 'aspiration', 'body-style', 'drive-wheels', 'engine-location', 'wheel-base', 'length', 'width', 'height', 'curb-weight', 'engine-type', 'num-of-cylinders', 'engine-size', 'fuel-system', 'bore', 'horsepower', 'peak-rpm', 'city-mpg', 'highway-mpg', 'price'], dtype='object')

# ONE_HOT_VECTOR

In [224]:
cars = pd.get_dummies(cars,drop_first=True)
cars.shape

(201, 76)

In [225]:
cars.head(100)

,normalized-losses,wheel-base,length,width,height,curb-weight,num-of-cylinders,engine-size,bore,horsepower,peak-rpm,city-mpg,highway-mpg,price,make_audi,...,drive-wheels_fwd,drive-wheels_rwd,engine-location_rear,engine-type_l,engine-type_ohc,engine-type_ohcf,engine-type_ohcv,engine-type_rotor,fuel-system_2bbl,fuel-system_4bbl,fuel-system_idi,fuel-system_mfi,fuel-system_mpfi,fuel-system_spdi,fuel-system_spfi
0,122.0,88.6,168.8,64.1,48.8,2548,4,130,3.470,111.0,5000.0,21,27,13495.0,0,...,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0
1,122.0,88.6,168.8,64.1,48.8,2548,4,130,3.470,111.0,5000.0,21,27,16500.0,0,...,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0
2,122.0,94.5,171.2,65.5,52.4,2823,6,152,2.680,154.0,5000.0,19,26,16500.0,0,...,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0
3,164.0,99.8,176.6,66.2,54.3,2337,4,109,3.190,102.0,5500.0,24,30,13950.0,1,...,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0
4,164.0,99.4,176.6,66.4,54.3,2824,5,136,3.190,115.0,5500.0,18,22,17450.0,1,...,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0
5,122.0,99.8,177.3,66.3,53.1,2507,5,136,3.190,110.0,5500.0,19,25,15250.0,1,...,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0
6,158.0,105.8,192.7,71.4,55.7,2844,5,136,3.190,110.0,5500.0,19,25,17710.0,1,...,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0
7,122.0,105.8,192.7,71.4,55.7,2954,5,136,3.190,110.0,5500.0,19,25,18920.0,1,...,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0
8,158.0,105.8,192.7,71.4,55.9,3086,5,131,3.130,140.0,5500.0,17,20,23875.0,1,...,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0
10,192.0,101.2,176.8,64.8,54.3,2395,4,108,3.500,101.0,5800.0,23,29,16430.0,0,...,0,1,0,0,1,0,0,0,0,0,0,0,1,0,0


# cleanign is completed

***********

In [157]:
cars.columns

Index(['normalized-losses', 'wheel-base', 'length', 'width', 'height', 'curb-weight', 'num-of-cylinders', 'engine-size', 'bore', 'horsepower', 'peak-rpm', 'city-mpg', 'highway-mpg', 'price', 'make_audi', 'make_bmw', 'make_chevrolet', 'make_dodge', 'make_honda', 'make_isuzu', 'make_jaguar', 'make_mazda',
       'make_mercedes-benz', 'make_mercury', 'make_mitsubishi', 'make_nissan', 'make_peugot', 'make_plymouth', 'make_porsche', 'make_renault', 'make_saab', 'make_subaru', 'make_toyota', 'make_volkswagen', 'make_volvo', 'fuel-type_audi', 'fuel-type_bmw', 'fuel-type_chevrolet', 'fuel-type_dodge', 'fuel-type_honda',
       'fuel-type_isuzu', 'fuel-type_jaguar', 'fuel-type_mazda', 'fuel-type_mercedes-benz', 'fuel-type_mercury', 'fuel-type_mitsubishi', 'fuel-type_nissan', 'fuel-type_peugot', 'fuel-type_plymouth', 'fuel-type_porsche', 'fuel-type_renault', 'fuel-type_saab', 'fuel-type_subaru', 'fuel-type_toyota',
       'fuel-type_volkswagen', 'fuel-type_volvo', 'aspiration_turbo', 'body-style

# normalization of data

In [226]:
df = cars.loc[:,:].copy()
for column in df.columns:
    df[column] = (df[column] - df[column].min()) / (df[column].max() - df[column].min())

#as price is also normalised above,we replace price column
df['price'] = cars['price']
df = df.head(250)

In [227]:
df.columns

Index(['normalized-losses', 'wheel-base', 'length', 'width', 'height', 'curb-weight', 'num-of-cylinders', 'engine-size', 'bore', 'horsepower', 'peak-rpm', 'city-mpg', 'highway-mpg', 'price', 'make_audi', 'make_bmw', 'make_chevrolet', 'make_dodge', 'make_honda', 'make_isuzu', 'make_jaguar', 'make_mazda',
       'make_mercedes-benz', 'make_mercury', 'make_mitsubishi', 'make_nissan', 'make_peugot', 'make_plymouth', 'make_porsche', 'make_renault', 'make_saab', 'make_subaru', 'make_toyota', 'make_volkswagen', 'make_volvo', 'fuel-type_audi', 'fuel-type_bmw', 'fuel-type_chevrolet', 'fuel-type_dodge', 'fuel-type_honda',
       'fuel-type_isuzu', 'fuel-type_jaguar', 'fuel-type_mazda', 'fuel-type_mercedes-benz', 'fuel-type_mercury', 'fuel-type_mitsubishi', 'fuel-type_nissan', 'fuel-type_peugot', 'fuel-type_plymouth', 'fuel-type_porsche', 'fuel-type_renault', 'fuel-type_saab', 'fuel-type_subaru', 'fuel-type_toyota',
       'fuel-type_volkswagen', 'fuel-type_volvo', 'aspiration_turbo', 'body-style

In [192]:
carsdd

,normalized-losses,wheel-base,length,width,height,curb-weight,num-of-cylinders,engine-size,bore,horsepower,peak-rpm,city-mpg,highway-mpg,price,make_audi,...,drive-wheels_fwd,drive-wheels_rwd,engine-location_rear,engine-type_l,engine-type_ohc,engine-type_ohcf,engine-type_ohcv,engine-type_rotor,fuel-system_2bbl,fuel-system_4bbl,fuel-system_idi,fuel-system_mfi,fuel-system_mpfi,fuel-system_spdi,fuel-system_spfi
0,122.0,88.6,168.8,64.1,48.8,2548,4,130,3.47,111.0,5000.0,21,27,13495.0,0,...,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0
1,122.0,88.6,168.8,64.1,48.8,2548,4,130,3.47,111.0,5000.0,21,27,16500.0,0,...,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0
2,122.0,94.5,171.2,65.5,52.4,2823,6,152,2.68,154.0,5000.0,19,26,16500.0,0,...,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0
3,164.0,99.8,176.6,66.2,54.3,2337,4,109,3.19,102.0,5500.0,24,30,13950.0,1,...,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0
4,164.0,99.4,176.6,66.4,54.3,2824,5,136,3.19,115.0,5500.0,18,22,17450.0,1,...,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
200,95.0,109.1,188.8,68.9,55.5,2952,4,141,3.78,114.0,5400.0,23,28,16845.0,0,...,0,1,0,0,1,0,0,0,0,0,0,0,1,0,0
201,95.0,109.1,188.8,68.8,55.5,3049,4,141,3.78,160.0,5300.0,19,25,19045.0,0,...,0,1,0,0,1,0,0,0,0,0,0,0,1,0,0
202,95.0,109.1,188.8,68.9,55.5,3012,6,173,3.58,134.0,5500.0,18,23,21485.0,0,...,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0
203,95.0,109.1,188.8,68.9,55.5,3217,6,145,3.01,106.0,4800.0,26,27,22470.0,0,...,0,1,0,0,1,0,0,0,0,0,1,0,0,0,0


# cross validation

In [290]:
#SPLITING DATA
df = df.sample(frac=1)
train_size = int(0.6 * len(cars))

train_set = df[:train_size]
test_set =  df[train_size:]

X_train = train_set.iloc[:,:-1]
X_test = test_set.iloc[:,:-1]

Y_train = train_set.price 
Y_test = test_set.price

In [291]:
# train_set.columns

In [292]:
def euclidean_distance(x,y):
    distance=np.sqrt(np.sum((np.array(x)-np.array(y))**2))
    return distance

In [293]:
def prediction(train_X, train_Y, terst_X, k):
    dist = []

    for i in range(len(train_X)):
        distances = euclidean_distance(train_X[i:i + 1], terst_X)
        dist.append(distances)
    dist = np.array(dist)
    
    #Sorting the distances
    distance = np.argsort(dist)[:k]
    tot = np.array([])

    for i in range(k):
        index = distance[i]
        y_val = train_Y[index:index + 1]
        tot = np.append(tot, np.array(y_val))

    #AVERAGE PRICE OF THE K NEAREST NEIGHBOURS
    avg = tot.sum() / k
    return avg

In [309]:
def rmse(train_X, test_X, k):
    rmse = 0
    
    train_Y=train_X[['price']]
    test_Y=test_X[['price']]
    for i in range(len(test_X)):
        #PREDICTED VALUE

        y_pred = prediction(train_X, train_Y, test_X[i:i + 1], k)

        error = ((y_pred - np.array(test_Y[i:i + 1])) ** 2)

        rmse += error

    # CALCULATING THE RMSE

    rmse = np.sqrt(rmse / len(test_X))
    
    return rmse

In [310]:
print('RMSE error with K=3:',rmse(train_set,test_set,3))

RMSE error with K=3: [[291.92039757]]


# 1)b)optimal k for this model using cross-validation 

# crossvalidation

In [311]:
#DIVIDING TRAININING DATA INTO 2 DATASETS

dataset_1=df.sample(frac=0.5)
dataset_2=df.drop(dataset_1.index)

#DIVIDING AGAIN 4 PARTS

ds11=dataset_1.sample(frac=0.5)
ds12=dataset_1.drop(ds11.index)
ds21=dataset_2.sample(frac=0.5)
ds22=dataset_2.drop(ds21.index)



In [312]:
# DEFINING CROSS VALIDATION FUNCTION

def cross_val(k):
    # CONCATENATING THREE DIFFERENT PARTS EACH TIME

    train1 = pd.concat([ds11, ds12, ds21])
    train2 = pd.concat([ds11, ds12, ds22])
    train3 = pd.concat([ds11, ds21, ds22])
    train4 = pd.concat([ds12, ds21, ds22])

    # GIVING THE CONCATED PART AS TRAIN AND THE LEFT PART AS TEST TO RMSE METHOD

    rmse_1 = rmse(train1,ds22, k)
    rmse_2 = rmse(train2,ds21, k)
    rmse_3 = rmse(train3,ds12, k)
    rmse_4 = rmse(train4,ds11, k)

    # CALCULATING THE AVERAGE RMSE VALUE
    avg_rmse = (rmse_1 + rmse_2 + rmse_3 + rmse_4) / 4
    return avg_rmse

In [313]:
# RUNNING THE CROSS VALIDATION FOR DIFFERENT VALUES OF K AND FINDING THE BEST K VALUE

k=0
high_rmse=1000000
for i in range(1,10):
    avgrmse=cross_val(i)
    print('rmse for k=',i,avgrmse)
    if avgrmse<high_rmse:
        high_rmsermse=avgrmse
        k=i
print('OPTIMAL K VALUE', k, ' RMSE VALUE IS', rmse)

rmse for k= 1 [[403.9386746]]
rmse for k= 2 [[457.89333262]]
rmse for k= 3 [[475.28762901]]
rmse for k= 4 [[500.80928812]]
rmse for k= 5 [[596.15236532]]
rmse for k= 6 [[650.72195704]]
rmse for k= 7 [[709.38063613]]
rmse for k= 8 [[723.04651974]]
rmse for k= 9 [[788.40194168]]
OPTIMAL K VALUE 9  RMSE VALUE IS <function rmse at 0x000002A82C96CCA0>
